# Research

In [1]:
import pandas as pd
import os

from typing import List

In [2]:
datal_files = os.listdir('data/intraday/')

In [3]:
def _check_nan(df: pd.DataFrame, columns: List[str]) -> bool:
    """ Checks if there are NaN values in the given columns of a dataframe. """
    return df.isnull().values.any()

def _check_missing_in_hourly(df: pd.DataFrame, column: str = None) -> bool:
    """ Checks if there are missing values in the given column of a dataframe. """
    if column is None:
        series = df.index
    else:
        series = df[column]
    return series.to_series().diff().dt.total_seconds().div(3600).round().eq(1).all()


In [4]:
def load_prices(path: str, check_missing: bool = False) -> pd.DataFrame:
    """ Loads hourly prices from csv file and returns a dataframe with hourly, daily and monthly prices."""
    # Import csv file
    prices_hourly = pd.read_csv(path)
    # Make column names lowercase
    prices_hourly.columns = prices_hourly.columns.str.lower()
    # Set date index
    prices_hourly = prices_hourly.set_index('date', inplace=False)
    prices_hourly.index = pd.to_datetime(prices_hourly.index)

    # Check for missing values
    if check_missing:
        if _check_nan(prices_hourly, ['open', 'high', 'low', 'close', 'volume']):
            raise ValueError("Missing values (NaN) in prices_hourly")
        if not _check_missing_in_hourly(prices_hourly):
            raise ValueError("Missing values in prices_hourly")

    # Convert to numeric
    prices_hourly = prices_hourly.astype({"open": float, "high": float, "low": float,
                                          "close": float, "volume": float,
                                          "name": str, "symbol": str})

    prices_daily = prices_hourly.resample('D').last()
    prices_monthly = prices_hourly.resample('M').last()

    return {"hourly": prices_hourly, "daily": prices_daily, "monthly": prices_monthly}

In [5]:
prices = load_prices('data/intraday/Bitcoin.csv')

In [6]:
import pandas as pd
from typing import Optional, List

def get_returns(prices: pd.DataFrame, lags: List[int], freq: str, columns: Optional[List[str]] = None,
                       normalize: bool = True, dropna: bool = False) -> pd.DataFrame:
    """ Calculates lagged returns from prices. The returns are optionally normalized with the geometric average """
    prices = prices.copy()
    # Set Columns to calculate the lagged returns for
    if columns is None:
        pass
        columns = ['high', 'low', 'open', 'close', 'volume']
    else:
        columns = [col.lower() for col in columns]

    # For each column calculate the lagged returns for the lags that were specified
    for column in columns:
        for lag in lags:
            prices[f'{column}_return_{lag}{freq}'] = prices[column].pct_change(lag)
            # Normalize returns
            if normalize:
                prices[f'{column}_return_{lag}{freq}'] = prices[f'{column}_return_{lag}{freq}'].add(1).pow(1 / lag).sub(1)

    # Drop NaNs
    if dropna:
        prices.dropna(inplace=True)
            
    return prices


def get_hourly_returns(prices: pd.DataFrame, lags: List[int], columns: Optional[List[str]] = None,
                       normalize: bool = True, dropna: bool = False) -> pd.DataFrame:
    """ Calculates hourly lagged returns from hourly prices."""
    return get_returns(prices, lags, 'h', columns, normalize, dropna)


def get_daily_returns(prices: pd.DataFrame, lags: List[int], columns: Optional[List[str]] = None,
                      normalize: bool = True, dropna: bool = False) -> pd.DataFrame:
    """ Calculates daily lagged returns from daily prices."""
    return get_returns(prices, lags, 'd', columns, normalize, dropna)


def get_monthly_returns(prices: pd.DataFrame, lags: List[int], columns: Optional[List[str]] = None,
                        normalize: bool = True, dropna: bool = False) -> pd.DataFrame:
    """ Calculates monthly lagged returns from monthly prices."""
    return get_returns(prices, lags, 'm', columns, normalize, dropna)


In [7]:
def get_momentums(prices: pd.DataFrame, lags: List[int], freq: str, columns: Optional[List[str]] = None,
                  normalize: bool = True, dropna: bool = False) -> pd.DataFrame:
    """ Calculates lagged momentums from prices."""
    prices = prices.copy()
    # Set Columns to calculate the lagged momentums for
    if columns is None:
        columns = ['high', 'low', 'open', 'close', 'volume']
    else:
        columns = [col.lower() for col in columns]



    # Drop NaNs
    if dropna:
        prices.dropna(inplace=True)

    return prices

,open,high,low,close,volume,symbol,name,open_return_1h,open_return_2h,open_return_3h,open_return_4h,open_return_5h
date,,,,,,,,,,,,
2017-08-17 04:00:00,4261.48,4313.62,4261.32,4308.83,47.181009,BTC,Bitcoin,NaN,NaN,NaN,NaN,NaN
2017-08-17 05:00:00,4308.83,4328.69,4291.37,4315.32,23.234916,BTC,Bitcoin,0.011111,NaN,NaN,NaN,NaN
2017-08-17 06:00:00,4330.29,4345.45,4309.37,4324.35,7.229691,BTC,Bitcoin,0.004980,0.016147,NaN,NaN,NaN
2017-08-17 07:00:00,4316.62,4349.99,4287.41,4349.99,4.443249,BTC,Bitcoin,-0.003157,0.001808,0.012939,NaN,NaN
2017-08-17 08:00:00,4333.32,4377.85,4333.32,4360.69,0.972807,BTC,Bitcoin,0.003869,0.000700,0.005684,0.016858,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-05 17:00:00,29032.72,29069.83,29031.39,29063.99,339.075680,BTC,Bitcoin,-0.000302,-0.001220,0.000223,0.000619,-0.000526
2023-08-05 18:00:00,29064.00,29099.00,29060.19,29076.86,352.453710,BTC,Bitcoin,0.001077,0.000775,-0.000144,0.001300,0.001697
2023-08-05 19:00:00,29076.86,29088.58,29066.00,29067.35,265.299380,BTC,Bitcoin,0.000442,0.001520,0.001218,0.000299,0.001743


## Package testing

In [8]:
from xtrader.dataloaders import ohlc
from xtrader.alphas import returns as re

In [9]:
prices = ohlc.load_prices('data/intraday/Bitcoin.csv', check_missing=False)
re.get_daily_returns(prices['daily'], [1, 2, 3], columns=['open'], normalize=True, dropna=True).head(6)

,open,high,low,close,volume,symbol,name,open_return_1d,open_return_2d,open_return_3d
date,,,,,,,,,,
2017-08-20,4151.14,4171.62,4069.13,4086.29,69.475467,BTC,Bitcoin,0.007368,0.005439,-0.012699
2017-08-21,4025.00,4031.87,3987.50,4016.00,10.963783,BTC,Bitcoin,-0.030387,-0.011690,-0.006648
2017-08-22,4009.10,4045.00,3944.03,4040.00,41.089781,BTC,Bitcoin,-0.003950,-0.017257,-0.009117
2017-08-23,4151.88,4169.99,4113.79,4114.01,41.651997,BTC,Bitcoin,0.035614,0.015639,0.000059
2017-08-24,4326.24,4371.68,4297.76,4316.01,35.775720,BTC,Bitcoin,0.041995,0.038800,0.024350
2017-08-25,4304.02,4343.18,4280.68,4280.68,9.873695,BTC,Bitcoin,-0.005136,0.018157,0.023943


In [10]:
import pandas_datareader.data as web
